# Washington State DOT Traffic API

source: https://wsdot.wa.gov/traffic/api/


In [1]:
import pandas as pd
import requests

from pathlib import Path

## Import env variables


In [2]:
from dotenv import load_dotenv
import os

# load the .env file
load_dotenv()

# access the env variables
api_key = os.getenv("API_KEY")

# print to verify
# print(f"API Key: {api_key}")

## URLs to Access API


In [3]:
TRAVEL_TIMES_URL = "http://wsdot.wa.gov/Traffic/api/TravelTimes/TravelTimesREST.svc/GetTravelTimesAsJson?AccessCode={ACCESSCODE}"

TRAFFIC_ALERTS_URL = "http://www.wsdot.wa.gov/Traffic/api/HighwayAlerts/HighwayAlertsREST.svc/GetAlertsAsJson?AccessCode={ACCESSCODE}"

WEATHER_INFORMATION_URL = "http://wsdot.wa.gov/Traffic/api/WeatherInformation/WeatherInformationREST.svc/GetCurrentWeatherInformationAsJson?AccessCode={ACCESSCODE}"

### Function to Get API data


In [12]:
def get_api_data(url, access_key):
    """_summary_

    Args:
        url (_type_): _description_
        access_key (_type_): _description_

    Returns:
        _type_: _description_
    """
    # create the url with the access key
    url_api = url.format(ACCESSCODE=access_key)
    response = requests.get(url_api)

    # check if request was successful
    if response.status_code == 200:
        print("Data fetched successfully.")
        data = response.json()
        df = pd.DataFrame(data)
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

    # return the dataframe and the status code
    return df, response.status_code

In [5]:
def get_updates(
    travel_times_url, traffic_alerts_url, weather_information_url, access_key
):

    # Get current time
    timestamp = pd.Timestamp.now()
    df_tt, response_tt = get_api_data(travel_times_url, access_key)
    df_ta, response_ta = get_api_data(travel_times_url, access_key)
    df_wa, response_wa = get_api_data(travel_times_url, access_key)

    # add pull_date_time as a foreign key 
    df_tt['timestamp'] = timestamp
    df_ta['timestamp'] = timestamp
    df_wa['timestamp'] = timestamp

    df_response = pd.DataFrame(
        [
            {
                "timestamp": timestamp,
                "travel_times_response": response_tt,
                "traffic_alerts_response": response_ta,
                "weather_alerts_response": response_wa,
            }
        ]
    )

    return df_response, df_tt, df_ta, df_wa

## Travel Times API


In [6]:
df_travel_times, reponse = get_api_data(TRAVEL_TIMES_URL, api_key)
df_travel_times.head()

Data fetched successfully.


,AverageTime,CurrentTime,Description,Distance,EndPoint,Name,StartPoint,TimeUpdated,TravelTimeID
0,25,25,Everett to Downtown Seattle using HOV lanes,26.72,{'Description': 'I-5 @ University St in Seattl...,Everett-Seattle HOV,"{'Description': 'I-5 @ 41st St in Everett', 'D...",/Date(1738215600000-0800)/,2
1,25,25,Downtown Seattle to Everett using HOV lanes,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett HOV,{'Description': 'I-5 @ University St in Seattl...,/Date(1738215600000-0800)/,3
2,25,25,Downtown Seattle to Everett,26.94,"{'Description': 'I-5 @ 41st St in Everett', 'D...",Seattle-Everett,{'Description': 'I-5 @ University St in Seattl...,/Date(1738215600000-0800)/,4
3,9,9,Downtown Bellevue to Issaquah,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738215600000-0800)/,5
4,9,9,Downtown Bellevue to Issaquah using HOV lanes,9.28,"{'Description': 'I-90 @ Front St in Issaquah',...",Bellevue-Issaquah HOV,{'Description': 'I-405 @ NE 8th St in Bellevue...,/Date(1738215600000-0800)/,6


## Traffic Alerts API


In [7]:
df_traffic_alerts, response = get_api_data(TRAFFIC_ALERTS_URL, api_key)
df_traffic_alerts.head()

Data fetched successfully.


,AlertID,County,EndRoadwayLocation,EndTime,EventCategory,EventStatus,ExtendedDescription,HeadlineDescription,LastUpdatedTime,Priority,Region,StartRoadwayLocation,StartTime
0,624112,Grant,"{'Description': None, 'Direction': 'B', 'Latit...",None,Rest Area,Open,,The blue Lake rest area is closed for the winter.,/Date(1731629066113-0800)/,Low,North Central,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1731628800000-0800)/
1,632850,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738328400000-0800)/,Maintenance,Open,,"Nightly from 9 p.m. until 5 a.m. Sunday, Jan. ...",/Date(1737765776500-0800)/,High,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737765300000-0800)/
2,632245,None,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738371600000-0800)/,Maintenance,Open,,"Weekdays, from 7 a.m. to 5 p.m., Wednesday, Ja...",/Date(1737148405100-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1737148080000-0800)/
3,632835,None,"{'Description': None, 'Direction': 'B', 'Latit...",None,Construction,Open,,Eastbound and westbound SR 18 will shift to th...,/Date(1737761204897-0800)/,Low,Northwest,"{'Description': None, 'Direction': 'B', 'Latit...",/Date(1738155600000-0800)/
4,632782,None,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1738152000000-0800)/,Construction,Open,,"Tuesday, Jan. 27 to the morning of Wednesday, ...",/Date(1737738428410-0800)/,Medium,Northwest,"{'Description': None, 'Direction': 'S', 'Latit...",/Date(1737738900000-0800)/


## Weather Information API


In [8]:
df_weather_alerts, response = get_api_data(WEATHER_INFORMATION_URL, api_key)
df_weather_alerts.head()

Data fetched successfully.


,BarometricPressure,Latitude,Longitude,PrecipitationInInches,ReadingTime,RelativeHumidity,SkyCoverage,StationID,StationName,TemperatureInFahrenheit,Visibility,WindDirection,WindDirectionCardinal,WindGustSpeedInMPH,WindSpeedInMPH
0,960.1,47.474800,-122.270400,None,/Date(1738215006000-0800)/,99.0,N/A,1909,S 144th St on SB I-5 at mp 155.32,36.68,1.0,165.0,SSE,0.0,0.0
1,1040.9,47.760633,-122.184048,None,/Date(1738215015000-0800)/,97.0,N/A,1910,NE 195th on SB I-405 at mp 24.58,30.92,1.0,5.0,N,0.0,0.0
2,984.4,47.509000,-121.885000,None,/Date(1738215007000-0800)/,82.0,N/A,1928,EB I-90 / SR-18 (Echo Lake) at mp 26.30,31.10,1.0,NaN,N/A,1.0,0.0
3,1006.5,47.726000,-122.324000,None,/Date(1738215003000-0800)/,97.0,N/A,1966,NE 130th Street on I-5 at mp 173.75,33.44,1.0,90.0,E,0.0,0.0
4,NaN,46.436000,-117.350000,None,/Date(1738215454000-0800)/,64.0,N/A,1968,Alpowa Summit on US 12 at mp 413.36,28.76,12.0,172.0,S,5.0,4.0


# Call to get all info from API


In [13]:
df_response, df_tt, df_ta, df_wa = get_updates(
    TRAVEL_TIMES_URL, TRAFFIC_ALERTS_URL, WEATHER_INFORMATION_URL, api_key
)

Data fetched successfully.
Data fetched successfully.
Data fetched successfully.


# Export to CSV


In [10]:
# Set path to datasets folder
datasets = Path("../datasets")

In [14]:
# Save dataframes to csv
df_response.to_csv(datasets / "api_fetch.csv", index=False)
df_tt.to_csv(datasets / "travel_times.csv", index=False)
df_ta.to_csv(datasets / "traffic_alerts.csv", index=False)
df_wa.to_csv(datasets / "weather_alerts.csv", index=False)

In [15]:
df_tt
df_response

,timestamp,travel_times_response,traffic_alerts_response,weather_alerts_response
0,2025-01-29 21:47:52.451024,200,200,200
